In [ ]:
import torch
import torch.nn as nn
from sklearn.datasets import load_wine
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tqdm

### dataset 

In [ ]:
data = load_wine(as_frame=True)

pandas_df = data['data']
targets= data['target']

display(pandas_df.head())
display(targets)

In [ ]:
scaler = StandardScaler()
df = scaler.fit_transform(pandas_df.to_numpy())

X_train, X_test, y_train, y_test = train_test_split(df, targets, test_size=0.3, shuffle=True)

In [ ]:
tensor_X_train = torch.from_numpy(X_train).to(torch.float32)
tensor_y_train = torch.from_numpy(y_train.to_numpy()).to(torch.int8)

display(tensor_X_train)
display(tensor_X_train.shape)
display(tensor_y_train)
display(tensor_y_train.shape)

In [ ]:
train_dataset = torch.utils.data.TensorDataset(tensor_X_train, tensor_y_train)
for i in train_dataset:
    print(i)
    break

In [ ]:
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=4, shuffle=True)
for i in train_dataloader:
    print(i)
    break

### model 

In [ ]:
model = nn.Sequential(nn.Linear(13, 32), 
                      nn.ReLU(), 
                      nn.Linear(32, 32),
                      nn.ReLU(), 
                      nn.Linear(32,3))

model

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
epochs = 50

for epoch in tqdm.tqdm(range(epochs)):
    for x, y in train_dataloader:
        outputs = model(x)
        tgs = y.to(torch.long)
        
        optimizer.zero_grad()
        loss = criterion(outputs, tgs)
        loss.backward()
        optimizer.step()

In [ ]:
model(train_dataset[0][0].unsqueeze(dim=0))

### eval 

In [ ]:
with torch.no_grad():
    result = model(torch.from_numpy(X_test).to(torch.float32))
    
result[0:5]

In [ ]:
torch.argmax(result, dim=1)

In [ ]:
torch.from_numpy(y_test.to_numpy()).to(torch.int64)

In [ ]:
torch.argmax(result, dim=1) == torch.from_numpy(y_test.to_numpy()).to(torch.int64)

# LeNet 

In [43]:
from torch.nn import Conv2d

In [44]:
conv2d = Conv2d

In [45]:
import torch.nn as nn

model = nn.Sequential(
    # C1 (1@32×32 → 6@28×28 → 6@14×14)
    nn.Conv2d(1, 6, 5),
    nn.Tanh(),
    nn.AvgPool2d(2, 2),

    # C3 (6@14×14 → 16@10×10 → 16@5×5)
    nn.Conv2d(6, 16, 5),
    nn.Tanh(),
    nn.AvgPool2d(2, 2),

	# C5 записанный в виде полносвязного слоя вместо свертки
    nn.Flatten(),
    nn.Linear(16 * 5 * 5, 120),
    nn.Tanh(),
    
    nn.Linear(120, 84),
    nn.Tanh(),
    nn.Linear(84, 10)
)

In [46]:
# !pip install torchsummary
from torchsummary import summary

In [47]:
summary(model, input_size=(1,32,32), device='cuda')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 28, 28]             156
              Tanh-2            [-1, 6, 28, 28]               0
         AvgPool2d-3            [-1, 6, 14, 14]               0
            Conv2d-4           [-1, 16, 10, 10]           2,416
              Tanh-5           [-1, 16, 10, 10]               0
         AvgPool2d-6             [-1, 16, 5, 5]               0
           Flatten-7                  [-1, 400]               0
            Linear-8                  [-1, 120]          48,120
              Tanh-9                  [-1, 120]               0
           Linear-10                   [-1, 84]          10,164
             Tanh-11                   [-1, 84]               0
           Linear-12                   [-1, 10]             850
Total params: 61,706
Trainable params: 61,706
Non-trainable params: 0
---------------------------------

# Torch | cv2 | PIL | convertations

In [22]:
# !pip install torchvision
import torch
import torch.nn as nn
import torchvision.transforms as T
from torchvision.transforms.functional import to_tensor

import numpy as np
import cv2 
from PIL import Image

In [23]:
cv_img_bgr = cv2.imread('jinx.jpg')
cv_img_rgb = cv2.cvtColor(cv_img_bgr, cv2.COLOR_BGR2RGB)
cv_img_rgb.shape

(2160, 3840, 3)

In [24]:
def rgb_array_to_tensor1(rgb: np.ndarray) -> torch.Tensor:
    # H, W, C -> C, H, W
    tensor = torch.from_numpy(rgb).permute(2, 0, 1).float()
    # Нормализуем и добавляем батч
    return (tensor / 255.0).unsqueeze(0) 

In [25]:
rgb_array_to_tensor1(cv_img_rgb).shape

torch.Size([1, 3, 2160, 3840])

In [26]:
i1 = rgb_array_to_tensor1(cv_img_rgb)

In [27]:
def rgb_array_to_tensor2(rgb: np.ndarray) -> torch.Tensor:
    """
    Преобразует RGB-изображение из numpy-массива в тензор PyTorch
    H,W,C -> C,H,W, нормализация к [0,1], добавление batch.
    """
    # permute + float + /255
    tensor = to_tensor(rgb)  # -> [3, H, W]
    # Добавляем batch dimension
    return tensor.unsqueeze(0)  # -> [1, 3, H, W]

In [28]:
rgb_array_to_tensor2(cv_img_rgb).shape

torch.Size([1, 3, 2160, 3840])

In [29]:
i2 = rgb_array_to_tensor2(cv_img_rgb)

In [30]:
(i1 == i2).all()

tensor(True)

In [32]:
def tensor_to_rgb_array(tensor: torch.Tensor) -> np.ndarray:
    
    if tensor.ndim == 4:
        tensor = tensor.squeeze(0)
    if tensor.ndim != 3 or tensor.shape[0] != 3:
        raise ValueError(f"Ожидается [3, H, W], получено: {tensor.shape}")

    tensor = tensor.mul(255.0)
    tensor = torch.clamp(tensor, 0, 255)
    tensor = tensor.to(torch.uint8)
    tensor = tensor.permute(1, 2, 0) # C,H,W -> H,W,C (RGB)
    return tensor.numpy()

In [38]:
pool = nn.AvgPool2d(kernel_size=11)

In [39]:
pooled_img = pool(i2)

In [40]:
pooled_img.shape

torch.Size([1, 3, 196, 349])

In [41]:
out_img = tensor_to_rgb_array(pooled_img)

In [42]:
Image.fromarray(out_img).show()